<기간별>
- 전기간 (whole)
    - 평균기온 민감(음/양)
    - 일강수량 민감(음/양)
    - 미세먼지 민감(음/양)
    - 진짜 민감한 상품들
    
    
- 따뜻한 달(warm), 시원한 달(cool)
    - 평균기온 민감(음/양)
    - 미세먼지 민감(음/양)
    - 진짜 민감한 상품들
    
    
- 장마기간(rainy)
    - 일강수량 민감(음/양)


- 2018년, 2019년
    - 평균기온 민감(음/양)
    - 일강수량 민감(음/양)
    - 미세먼지 민감(음/양)
    - 진짜 민감한 상품들



In [561]:
import os
import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import warnings
%matplotlib inline

rc('font', family='AppleGothic')
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False

warnings.filterwarnings(action='ignore')
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [605]:
# 일별
# data = pd.read_csv('./raw_data/final_day.csv', index_col = 0)
data = pd.read_csv('./target.csv', index_col=0)

In [606]:
# # 주별
# data = pd.read_csv('./raw_data/final_week.csv', index_col = 0)

# # 일사량 제거
# data.drop('일사량', axis=1, inplace=True)

holiday_product = ['과일세트', '곶감/반건시', '차 선물세트', '혼합견과', '홍삼/인삼 제품', '굴비/조기', '표고버섯', '한우육', '소고기 등심/안심', '홍삼절편/홍삼정과', '홍삼액/홍삼정']
data.drop(holiday_product, axis=1, inplace=True)

In [608]:
weathers = ['평균기온', '일강수량', '미세먼지'] # 독립 변수
products = data.columns[3:].to_list() # 종속 변수

In [610]:
big_cat = pd.read_csv('./대분류.csv', index_col=0)
big_cat = big_cat[big_cat['sm_cat'].isin(products)]

In [697]:
pd.read_csv('./result01 (1).csv', index_col=0).head(10)

,OLS,LSTM,RETAIN,GRU,VAR,recommended model
기능성 트러블케어 화장품,173459.0426,0.0548,226549.3760,0.0626,5030996.7333,LSTM
기능성 화이트닝 화장품,44.9475,0.0376,37.0872,0.0244,173877.5481,GRU
기능성 화장품 세트,39095.0043,0.0485,55044.9161,0.0451,564342637.6028,GRU
기초 화장용 로션,10423371.8678,0.0069,1204233.2551,0.0071,NaN,LSTM
기초 화장용 미스트,941085.6004,0.0720,668352.3953,0.0770,226038273.5154,LSTM
기초 화장용 스킨,11665702.8723,0.0106,4524818.7272,0.0085,238155076171.5057,GRU
기초 화장용 에센스,39538739.5622,0.0512,58967986.3843,0.0769,5605589053.4425,LSTM
기초 화장용 오일/앰플,10545596.5348,0.0447,15595340.8923,0.0845,859024330.5349,LSTM
기초 화장용 크림,284415955.9047,0.0318,131997328.6664,0.0433,NaN,LSTM
남성 로션,196077.6081,0.0113,110517.0948,0.0182,5856717731.5890,LSTM


# 스케일링
### 판매량을 0 ~ 100 개로 맞춤

In [611]:
from sklearn.preprocessing import MinMaxScaler

def MMscale(data):
    df = data.copy()
    
    buy = df.iloc[:, 3:]
    weather = df.iloc[:, :3]
    
    scaler = MinMaxScaler()
    scaled_buy = scaler.fit_transform(buy)
    
    scaled_buy = pd.DataFrame(scaled_buy, columns= buy.columns, index=buy.index)
    scaled_buy = scaled_buy * 100
    df = pd.concat([weather, scaled_buy], axis=1)
    
    return df

# OLS 모델링

In [612]:
import statsmodels.api as sm
from statsmodels.stats.stattools import durbin_watson
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error, r2_score

def OLS_(data, cat):
    X = data[weathers]
    y = data[cat]
    
    model = sm.OLS(y, X).fit()
    mse = mean_squared_error(y_true=y, y_pred = model.predict(X))

    return model, model.rsquared_adj, mse, durbin_watson(model.resid) 
    # 모델, R제곱, mse, DW지수

In [613]:
def final_score(df):
    score = pd.DataFrame(index=['평균기온', '일강수량', '미세먼지', 'adjusted R squared', 'Durbin-Watson'])

    for cat in products:
        ols_model, ols_adjr2, ols_mse, DW = OLS_(df, cat)

        if ols_adjr2 < 0.13:
            print(cat, 'OLS 모델 설명력 낮음 :', ols_adjr2)

        coef = pd.DataFrame(ols_model.params, columns=['coef'])
        pvalue = pd.DataFrame(ols_model.pvalues, columns=['pvalue'])
        temp = pd.concat([coef, pvalue], axis=1)

        idx = temp[temp.pvalue > 0.05].index # 유의확률 기준 초과하는 변수 (고려하지 않음)
        temp.drop('pvalue', axis=1, inplace=True)
        temp.rename(columns={'coef':cat}, inplace=True)
        temp.loc[idx] = np.nan
        temp.loc['adjusted R squared'] = ols_adjr2
        temp.loc['Durbin-Watson'] = DW

        score = pd.concat([score, temp], axis=1)
    
    return score

## 비교하기 클래스

- 객체 생성시 df(score 데이터프레임)과 기준으로 삼을 weather(날씨) 입력
- power : order가 'plus' 일 때 양의 상관관계가 강한 순으로 정렬, 'minus' 일 때 음의 상관관계가 강한 순으로 정렬, 'neutral'일 때 상관관계가 강한 순으로 정렬
- big : power 기능에 대분류 필터링을 추가함 ('가전' or '냉난방가전', '뷰티', '식품')

In [614]:
class Comparison:
    def __init__(self, df, weather):
        self.df = df
        self.df_T = self.df.transpose()
        self.weather = weather

        self.plus = self.df_T[self.df_T[self.weather] > 0].index.to_list()
        self.minus = self.df_T[self.df_T[self.weather] < 0].index.to_list()
    
    def power(self, order ='plus'): # 음/양에 따라 내림차순 정렬
        if order == 'plus':
            return self.df_T[self.df.columns.isin(self.plus)].sort_values(by=self.weather, ascending=False)
        elif order == 'minus':
            return self.df_T[self.df.columns.isin(self.minus)].sort_values(by=self.weather, ascending=True)
        elif order == 'neutral':
            return abs(self.df_T).sort_values(by=self.weather, ascending=False)
        
    def big(self, cat, order ='plus'): # 대분류 필터링 + 음/양에 따라 내림차순 정렬
        if cat == '가전':
            cat = '냉난방가전'
        reset = self.df_T.reset_index().rename(columns={'index':'sm_cat'})
        merged = pd.merge(reset, big_cat, how='outer', on='sm_cat')
        by_cat = merged[merged['big_cat']==cat]
        if order == 'plus':
            return by_cat.sort_values(by=self.weather, ascending=False)
        elif order == 'minus':
            return by_cat.sort_values(by=self.weather, ascending=True)

# 전 기간

In [615]:
# 1. 스케일링
whole = data.copy()
whole = MMscale(whole)

In [616]:
# 2. 모델링 및 점수 기록
score_whole = final_score(whole)

In [617]:
# 3. 비교 객체 생성

temper_whole = Comparison(score_whole, '평균기온')
rain_whole = Comparison(score_whole, '일강수량')
dust_whole = Comparison(score_whole, '미세먼지')

In [618]:
# 미세먼지와 양의 상관관계인 상품 리스트
print(temper_whole.minus)

['핸드크림', '랍스타', '가열식 가습기', '코코아/핫초코', '히터', '복합식 가습기', '자연식 가습기', '초음파식 가습기', '색조 메이크업 립밤', '유자차', '율무차', '공기청정기', '공기정화 용품', '해초류 ', '감귤/한라봉/오렌지', '굴 생물', '온풍기', '라디에이터', '에어워셔', '컨벡터']


In [619]:
temper_whole.power('plus').head(10)

,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson
참외/메론/수박,1.3078,-0.1310,NaN,0.6175,0.3171
이온음료,1.2087,-0.1023,0.2747,0.8161,0.6847
바디케어용 제모제,1.1597,NaN,0.2520,0.7429,0.5378
계란,1.1399,0.2328,0.4261,0.8032,0.6969
다이어트용 헬스보충식품,1.1312,NaN,0.4745,0.7947,0.7003
벽걸이형 선풍기,1.1119,-0.1647,NaN,0.5824,0.5311
뷰티 타투,1.0996,NaN,0.0524,0.6988,0.8122
애프터선,1.0780,NaN,0.1703,0.7386,0.8765
김치류,1.0635,0.1108,0.3165,0.7650,1.1114
탄산수,1.0463,NaN,0.4839,0.8126,0.7090


In [620]:
temper_cool.big('식품', 'minus').head(20)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
40,딸기/복분자/블루베리,-1.4524,0.4873,1.1375,0.7332,0.7362,식품
68,유제품 음료,-0.7715,0.3691,0.4803,0.4027,0.2276,식품
118,해초류,-0.4620,0.1925,0.2879,0.3679,1.4523,식품
11,전복 생물,-0.2633,NaN,0.2658,0.3686,0.8558,식품
81,인삼/수삼/산삼,-0.2324,NaN,0.3956,0.5253,0.6907,식품
83,양념우육,0.1714,NaN,0.2593,0.5981,0.6923,식품
73,레몬/자몽,0.2036,NaN,0.2756,0.6473,1.1445,식품
18,더치커피,0.2438,0.1377,0.2805,0.6047,1.1603,식품
158,건자두,0.2655,NaN,0.5316,0.6644,0.8021,식품
60,새우/대하,0.2736,NaN,0.2801,0.5951,1.3380,식품


In [621]:
rain_rainy.power(order='plus').head(10)

,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson
제습기,1.3429,0.6351,NaN,0.6895,0.8371
공기정화 용품,NaN,0.4765,1.2144,0.8657,1.3832
낙지,NaN,0.4552,NaN,0.3224,1.9181
바디 세트,NaN,0.4219,NaN,0.5942,1.7037
선케어용 선밤,NaN,0.3854,NaN,0.7878,1.8410
온풍기,NaN,0.3534,NaN,0.5106,2.0147


In [622]:
dust_cool.power().head(10)

,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson
딸기/복분자/블루베리,-1.4524,0.4873,1.1375,0.7332,0.7362
시금치,0.7250,0.4856,1.0005,0.7464,0.8082
생수,1.1924,NaN,0.9902,0.8014,0.7030
파/양파,1.2972,0.3332,0.9768,0.7699,0.5870
식혜/수정과,NaN,0.4734,0.9520,0.7359,0.9313
뷰티용 뷰러,-0.5159,0.4502,0.9512,0.7516,0.9436
고추/피망/파프리카,1.2184,NaN,0.9400,0.7458,0.6004
둥굴레차,NaN,NaN,0.9381,0.6809,0.9064
네일관리 소품,NaN,0.2886,0.9118,0.7576,1.0585
탄산음료,NaN,0.4627,0.8696,0.7172,0.4401


# 4 ~ 9 월, 10 ~ 3월

In [623]:
# 1. 데이터 구축

if len(data) == 730:
    data.index = pd.to_datetime(data.index)
    warm = data[data.index.month.isin([4, 5, 6, 7, 8, 9])]
    cool = data[data.index.month.isin([10, 11, 12, 1, 2, 3])]
elif len(data) == 105:
    hot = pd.concat([data.iloc[13:39], data.iloc[65:91]])
    cool = pd.concat([data.iloc[:13], data.iloc[39:65], data.iloc[91:]])
else: 
    print('데이터 개수 확인바람')

In [624]:
# 2. 스케일링

warm = MMscale(warm)
cool = MMscale(cool)

In [625]:
# 3. 모델링 및 점수 기록

score_warm = final_score(warm)
score_cool = final_score(cool)

온수매트 OLS 모델 설명력 낮음 : 0.11120410000452685
카페트매트 OLS 모델 설명력 낮음 : 0.09291889530215713


In [626]:
# 4. 설명 객체

temper_warm = Comparison(score_warm, '평균기온')
rain_warm = Comparison(score_warm, '일강수량')
dust_warm = Comparison(score_warm, '미세먼지')

temper_cool = Comparison(score_cool, '평균기온')
rain_cool = Comparison(score_cool, '일강수량')
dust_cool = Comparison(score_cool, '미세먼지')

In [627]:
rain_warm.power(order='minus').head(10)

,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson
냉풍기,0.7560,-0.2734,-0.1838,0.3671,0.3614
업소용 선풍기,1.1012,-0.2628,-0.2232,0.5514,0.6307
이동형 에어컨,0.8922,-0.2469,-0.1680,0.4979,0.4345
휴대용 선풍기,0.7474,-0.2119,-0.0803,0.5811,0.5363
에어컨 리모컨,1.1971,-0.1979,-0.2520,0.6181,0.4843
이온음료,1.2685,-0.1918,0.2676,0.8349,0.7286
벽걸이형 선풍기,1.1430,-0.1848,NaN,0.6333,0.5769
벽걸이 에어컨,0.5293,-0.1803,0.1746,0.5276,0.3885
멀티형 에어컨,NaN,-0.1642,0.4078,0.4095,1.1003
스탠드형 냉온풍기,0.3388,-0.1005,0.1670,0.6015,1.0821


In [628]:
temper_cool.big('식품', 'minus').head(20)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
38,딸기/복분자/블루베리,-1.4524,0.4873,1.1375,0.7332,0.7362,식품
65,유제품 음료,-0.7715,0.3691,0.4803,0.4027,0.2276,식품
110,해초류,-0.4620,0.1925,0.2879,0.3679,1.4523,식품
11,전복 생물,-0.2633,NaN,0.2658,0.3686,0.8558,식품
76,인삼/수삼/산삼,-0.2324,NaN,0.3956,0.5253,0.6907,식품
78,양념우육,0.1714,NaN,0.2593,0.5981,0.6923,식품
70,레몬/자몽,0.2036,NaN,0.2756,0.6473,1.1445,식품
18,더치커피,0.2438,0.1377,0.2805,0.6047,1.1603,식품
147,건자두,0.2655,NaN,0.5316,0.6644,0.8021,식품
57,새우/대하,0.2736,NaN,0.2801,0.5951,1.3380,식품


# 장마기간

- 2018년 : 6/19 ~ 7/11
- 2019년 : 6/26 ~ 7/29

In [630]:
# 1. 데이터 구축

import datetime

if len(data) == 730:
    date_list1 = [(datetime.date(2018, 6, 19) + datetime.timedelta(days = day)).isoformat() for day in range(23)]
    date_list2 = [(datetime.date(2019, 6, 26) + datetime.timedelta(days = day)).isoformat() for day in range(34)]
    
    rainy = data[data.index.isin(date_list1 + date_list2)]
    
elif len(data) == 105:
    rainy = pd.concat([data.iloc[24:28], data.iloc[77:82]])
    
else:
    print('데이터 개수 확인 바람')

In [631]:
# 2. 스케일링

rainy = MMscale(rainy)

In [632]:
# 3. 모델링 및 점수 기록

score_rainy = final_score(rainy)

견과류 잣/은행 OLS 모델 설명력 낮음 : 0.05352552478363748


In [633]:
# 4. 설명 객체

temper_rainy = Comparison(score_rainy, '평균기온')
rain_rainy = Comparison(score_rainy, '일강수량')
dust_rainy = Comparison(score_rainy, '미세먼지')

In [634]:
rain_rainy.power(order='plus').head(10)

,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson
제습기,1.3429,0.6351,NaN,0.6895,0.8371
공기정화 용품,NaN,0.4765,1.2144,0.8657,1.3832
낙지,NaN,0.4552,NaN,0.3224,1.9181
바디 세트,NaN,0.4219,NaN,0.5942,1.7037
선케어용 선밤,NaN,0.3854,NaN,0.7878,1.8410
온풍기,NaN,0.3534,NaN,0.5106,2.0147


In [635]:
rain_rainy.power(order='minus').head(10)

,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson
계란,2.6169,-0.4460,-1.1117,0.6685,1.3697


# 미세먼지 고농도 시기

## 12~3월

In [636]:
# 1. 데이터 구축

data.index = pd.to_datetime(data.index)
dusty = data[data.index.month.isin([1, 2, 3, 12])]

In [637]:
# 2. 스케일링

dusty = MMscale(dusty)

In [638]:
# 3. 모델링 및 점수 기록

score_dusty = final_score(dusty)

# 2018년, 2019년

In [639]:
# 1. 데이터 구축

year2018 = data[data.index.year.isin([2018])]
year2019 = data[data.index.year.isin([2019])]

In [640]:
# 2. 스케일링

year2018 = MMscale(year2018)
year2019 = MMscale(year2019)

In [641]:
# 3. 모델링 및 점수 기록

score_2018 = final_score(year2018)
score_2019 = final_score(year2019)

랍스타 OLS 모델 설명력 낮음 : 0.11482590667894355
스킨케어 수면팩 OLS 모델 설명력 낮음 : 0.12273407827751603
베이스 메이크업용 메이크업베이스 OLS 모델 설명력 낮음 : 0.1271682970378757


In [642]:
# 4. 설명 객체

temper_2018 = Comparison(score_2018, '평균기온')
rain_2018 = Comparison(score_2018, '일강수량')
dust_2018 = Comparison(score_2018, '미세먼지')

temper_2019 = Comparison(score_2019, '평균기온')
rain_2019 = Comparison(score_2019, '일강수량')
dust_2019 = Comparison(score_2019, '미세먼지')

In [643]:
temper_2018.power().head(10)

,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson
아이스티,1.5966,NaN,0.3704,0.8105,0.5572
미숫가루/곡물가루,1.5022,NaN,0.2075,0.8187,1.0541
참외/메론/수박,1.4993,-0.1934,NaN,0.6612,0.2790
오이/가지,1.3556,0.3605,0.7136,0.8435,1.0110
이온음료,1.3432,NaN,0.2268,0.7994,0.6842
바디케어용 제모제,1.2979,NaN,0.2721,0.7550,0.5671
뷰티 타투,1.2574,NaN,0.0926,0.7254,0.6904
탁상/USB 선풍기,1.2305,-0.2046,NaN,0.5838,0.4347
계란,1.1729,0.4149,0.3684,0.7999,0.6994
벽걸이형 선풍기,1.1647,-0.1831,NaN,0.5844,0.5450


In [644]:
temper_2019.power().head(10)

,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson
참외/메론/수박,1.6677,NaN,NaN,0.6548,0.3857
다이어트용 헬스보충식품,1.5078,NaN,0.4262,0.8241,0.9307
휴대용 선풍기,1.4936,NaN,-0.1310,0.5812,0.3924
이온음료,1.4309,-0.1848,0.4064,0.8277,0.7176
핸드워시/스크럽,1.3747,NaN,0.5019,0.8108,0.8965
김치류,1.3516,NaN,0.3477,0.7903,1.1051
네일컬러,1.3452,NaN,0.2653,0.7751,1.1184
뷰티 타투,1.3218,NaN,NaN,0.7019,0.9655
탄산수,1.3035,NaN,0.3638,0.8051,0.8254
바디케어용 제모제,1.2681,NaN,0.2542,0.7274,0.5348


In [645]:
score_whole.to_csv('./result_전기간 모델링 결과.csv')
score_warm.to_csv('./result_따뜻한 달 모델링 결과.csv')
score_cool.to_csv('./result_시원한 달 모델링 결과.csv')
score_rainy.to_csv('./result_장마철 모델링 결과.csv')
score_2018.to_csv('./result_2018년 모델링 결과.csv')
score_2019.to_csv('./result_2019년 모델링 결과.csv')

In [646]:
score_dusty.to_csv('./result_미세먼지 고농도 모델링 결과.csv')

------------------

------------------

# 성별

In [647]:
raw = pd.read_csv('./raw_data/buy.csv', index_col=0)

In [648]:
def buy_gender(data, sex):
    df = data.copy()
    df = df[df['sex']==sex]
    df.drop(['sex', 'age', 'big_cat'], axis=1, inplace=True)
    df = df.groupby(['date', 'sm_cat']).sum()
    df = df.stack().reset_index()
    df.drop('level_2', axis=1, inplace=True)
    df.columns = ['date', 'sm_cat', 'qty']
    df = df.pivot_table('qty', 'date', 'sm_cat')
    df.fillna(0, inplace=True)
    df.index = pd.to_datetime(df.index, format='%Y%m%d')
    return df

In [649]:
woman = buy_gender(raw, 'F')
man = buy_gender(raw, 'M')

In [650]:
woman = pd.concat([data.iloc[:, :4], woman], axis=1)
man = pd.concat([data.iloc[:, :4], man], axis=1)

In [651]:
man

,평균기온,일강수량,미세먼지,장어,가공란,가스온수기,가열식 가습기,가자미,갈비/찜/바비큐용 돈육,갈비용 우육,...,홍어,홍차,화장 비누,화장 퍼프,환자식,황토매트,회,휴대용 선풍기,흰우유,히터
date,,,,,,,,,,,,,,,,,,,,,
2018-01-01,-0.2189,0.0000,32.9630,18,25.0000,0.0000,3.0000,7.0000,8.0000,0.0000,...,2.0000,12.0000,74.0000,14.0000,15.0000,0.0000,333.0000,0.0000,190.0000,59.0000
2018-01-02,0.3330,0.0000,40.0370,24,36.0000,0.0000,9.0000,3.0000,3.0000,0.0000,...,2.0000,24.0000,102.0000,23.0000,11.0000,2.0000,333.0000,2.0000,237.0000,140.0000
2018-01-03,-1.3979,1.9000,23.1852,24,34.0000,0.0000,4.0000,7.0000,1.0000,1.0000,...,0.0000,13.0000,98.0000,16.0000,14.0000,3.0000,371.0000,0.0000,211.0000,149.0000
2018-01-04,-1.5768,1.9167,26.4231,29,21.0000,0.0000,6.0000,3.0000,2.0000,0.0000,...,5.0000,14.0000,96.0000,27.0000,24.0000,4.0000,442.0000,1.0000,203.0000,121.0000
2018-01-05,0.0200,1.1600,29.6429,21,21.0000,0.0000,8.0000,4.0000,3.0000,1.0000,...,2.0000,11.0000,55.0000,22.0000,12.0000,2.0000,433.0000,0.0000,193.0000,128.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,0.7063,0.5357,22.6552,59,56.0000,1.0000,7.0000,3.0000,3.0000,6.0000,...,11.0000,19.0000,82.0000,15.0000,28.0000,3.0000,560.0000,0.0000,225.0000,117.0000
2019-12-28,0.6221,0.5000,32.8621,35,40.0000,0.0000,10.0000,21.0000,15.0000,7.0000,...,3.0000,15.0000,67.0000,19.0000,22.0000,2.0000,620.0000,3.0000,218.0000,33.0000
2019-12-29,2.8537,7.4067,34.1724,25,49.0000,0.0000,2.0000,13.0000,9.0000,8.0000,...,1.0000,9.0000,60.0000,32.0000,13.0000,2.0000,583.0000,0.0000,260.0000,56.0000


## 여성

In [652]:
final_woman = pd.DataFrame(index=['평균기온', '일강수량', '미세먼지', 'adjusted R squared'])

for cat in products:
    ols_model, ols_adjr2, ols_mse, DW = OLS_(woman, cat)
    
    if ols_adjr2 < 0.13:
        print(cat, 'OLS 모델 설명력 낮음 :', ols_adjr2)
    
    coef = pd.DataFrame(ols_model.params, columns=['coef'])
    pvalue = pd.DataFrame(ols_model.pvalues, columns=['pvalue'])
    temp = pd.concat([coef, pvalue], axis=1)
    
    idx = temp[temp.pvalue > 0.05].index # 유의확률 기준 초과하는 변수 (고려하지 않음)
    temp.drop('pvalue', axis=1, inplace=True)
    temp.rename(columns={'coef':cat}, inplace=True)
    temp.loc[idx] = np.nan
    temp.loc['adjusted R squared'] = ols_adjr2
    temp.loc['Durbin-Watson'] = DW
    
    final_woman = pd.concat([final_woman, temp], axis=1)

ValueError: shapes (730,2) and (730,2) not aligned: 2 (dim 1) != 730 (dim 0)

In [ ]:
final_man = pd.DataFrame(index=['평균기온', '일강수량', '미세먼지', 'adjusted R squared'])

for cat in products:
    ols_model, ols_adjr2, ols_mse, DW = OLS_(man, cat)
    
    if ols_adjr2 < 0.13:
        print(cat, 'OLS 모델 설명력 낮음 :', ols_adjr2)
    
    coef = pd.DataFrame(ols_model.params, columns=['coef'])
    pvalue = pd.DataFrame(ols_model.pvalues, columns=['pvalue'])
    temp = pd.concat([coef, pvalue], axis=1)
    
    idx = temp[temp.pvalue > 0.05].index # 유의확률 기준 초과하는 변수 (고려하지 않음)
    temp.drop('pvalue', axis=1, inplace=True)
    temp.rename(columns={'coef':cat}, inplace=True)
    temp.loc[idx] = np.nan
    temp.loc['adjusted R squared'] = ols_adjr2
    temp.loc['Durbin-Watson'] = DW
    
    final_man = pd.concat([final_man, temp], axis=1)

In [ ]:
final

In [ ]:
final_man

# 나이

In [ ]:
def buy_age(data, age):
    df = data.copy()
    df = df[df['age']==age]
    df.drop(['sex', 'age', 'big_cat'], axis=1, inplace=True)
    df = df.groupby(['date', 'sm_cat']).sum()
    df = df.stack().reset_index()
    df.drop('level_2', axis=1, inplace=True)
    df.columns = ['date', 'sm_cat', 'qty']
    df = df.pivot_table('qty', 'date', 'sm_cat')
    df.fillna(0, inplace=True)
    df.index = pd.to_datetime(df.index, format='%Y%m%d')
    return df

In [ ]:
age20 = buy_age(raw, 20)
age30 = buy_age(raw, 30)
age40 = buy_age(raw, 40)
age50 = buy_age(raw, 50)
age60 = buy_age(raw, 60)

In [ ]:
age20 = pd.concat([data.iloc[:, :4], age20], axis=1)
age30 = pd.concat([data.iloc[:, :4], age30], axis=1)
age40 = pd.concat([data.iloc[:, :4], age40], axis=1)
age50 = pd.concat([data.iloc[:, :4], age50], axis=1)
age60 = pd.concat([data.iloc[:, :4], age60], axis=1)

In [ ]:
def age_func(data):
    final_df = pd.DataFrame(index=['평균기온', '일강수량', '일사량', '미세먼지', 'adjusted R squared'])

    for cat in products:
        ols_model, ols_adjr2, ols_mse, DW = OLS_(data, cat)

        if ols_adjr2 < 0.13:
            print(cat, 'OLS 모델 설명력 낮음 :', ols_adjr2)

        coef = pd.DataFrame(ols_model.params, columns=['coef'])
        pvalue = pd.DataFrame(ols_model.pvalues, columns=['pvalue'])
        temp = pd.concat([coef, pvalue], axis=1)

        idx = temp[temp.pvalue > 0.05].index # 유의확률 기준 초과하는 변수 (고려하지 않음)
        temp.drop('pvalue', axis=1, inplace=True)
        temp.rename(columns={'coef':cat}, inplace=True)
        temp.loc[idx] = np.nan
        temp.loc['adjusted R squared'] = ols_adjr2
    temp.loc['Durbin-Watson'] = DW


        final_df = pd.concat([final_df, temp], axis=1)
    return final_df

In [ ]:
final_20 = age_func(age20)
final_30 = age_func(age30)
final_40 = age_func(age40)
final_50 = age_func(age50)

In [ ]:
age60['색조 메이크업 세트'] = 0
age60['여성향수'] = 0
final_60 = age_func(age60)

In [ ]:
final_60